In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *

#### Data read and pre-processing

In [6]:
PATH='data/nietzsche/'

In [7]:
text=open(f'{PATH}nietzsche.txt').read()
len(text)

600893

In [5]:
text[:200]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrib'

In [8]:
chars=sorted(list(set(text)))
vocab_size=len(chars)+1
vocab_size

85

In [9]:
chars.insert(0,'\0')
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [10]:
stoi={c:i for i,c in enumerate(chars)}
itos={i:c for i,c in enumerate(chars)}

In [11]:
itos.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84])

In [12]:
idx=[stoi[char] for char in text]
len(idx)

600893

In [13]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

#### Basic RNN model object

In [12]:
n_hidden=256

In [13]:
n_fac=42

In [14]:
cs=8

In [15]:
range(cs)
range(len(idx)-cs)

[[print(i,j,i+j) for i in range(3)] for j in range(5)]

0 0 0
1 0 1
2 0 2
0 1 1
1 1 2
2 1 3
0 2 2
1 2 3
2 2 4
0 3 3
1 3 4
2 3 5
0 4 4
1 4 5
2 4 6


[[None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [16]:
c_in_dat=[[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs) ]

In [17]:
c_out_dat=[idx[j+cs] for j in range(len(idx)-cs)]

In [18]:
xs=np.stack(c_in_dat,axis=0)
xs.shape

(600885, 8)

In [19]:
xs[0]

array([40, 42, 29, 30, 25, 27, 29,  1])

In [20]:
y=np.stack(c_out_dat,axis=0)

In [21]:
val_idx=get_cv_idxs(len(idx)-cs-1)

In [21]:
type(cs)

int

In [22]:
md=ColumnarModelData.from_arrays('.',val_idx,xs,y,bs=512)

In [24]:
class FirstRNN(nn.Module):
    def __init__(self,vocab_size,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        self.l_in=nn.Linear(n_fac,n_hidden)
        self.l_hidden=nn.Linear(n_hidden,n_hidden)
        self.l_out=nn.Linear(n_hidden,vocab_size)
        
    def forward(self,*cs):
        bs=cs[0].size(0)
        h=V(torch.zeros(bs,n_hidden).cuda())
        for c in cs:
            inp=F.relu(self.l_in(self.e(c)))
            h=F.tanh(self.l_hidden(h+inp))
        return(F.log_softmax(self.l_out(h),dim=-1))
        
    

In [26]:
m=FirstRNN(vocab_size,n_fac).cuda()
opt=optim.Adam(m.parameters(),1e-2)

In [27]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [41]:
xs[0]


 56
 68
  2
 56
 65
 58
 68
 74
 62
 57
 73
 62
 69
 60
 69
 59
 73
 72
 71
 68
 72
 54
 65
 68
  2
 29
 62
 62
 74
 67
 68
 72
 65
 69
  2
 61
 71
 78
 73
  2
 54
 54
  2
 71
 58
 72
 55
 73
 54
 73
 59
 62
 58
 72
  2
  9
 71
 58
 67
 78
 65
 65
 72
 73
 65
 67
  2
 61
 73
 67
 65
 67
 58
 67
 72
 33
 39
 31
 69
 54
  2
 67
  2
 57
  2
 67
  1
 66
  2
  2
 65
 61
 71
 56
 62
 73
 73
  8
 54
  2
  2
 68
  2
 58
 73
 54
 54
 74
 73
 61
  8
 68
  2
  2
 57
 66
 58
 54
 27
 73
 73
 78
 58
 67
 73
 59
 54
 62
 54
  2
 56
 67
 68
 56
 68
 73
 58
 72
 66
  2
 65
 62
 58
 68
 58
 69
 57
 72
 58
  2
 71
 58
  8
 58
  4
 67
 59
 67
 58
  2
 72
 73
 62
 68
 66
 61
  2
 72
 76
 58
 73
 65
  2
 73
 73
 58
 68
 62
  2
 76
  2
  2
 57
 73
 67
 60
 68
  2
 74
  2
 73
  8
 65
 69
 73
 68
 58
 72
 62
 73
 67
 62
 67
 58
  8
 67
 39
  2
 62
 54
  2
  2
 58
 68
 66
 57
 71
  2
  8
 58
 65
  1
 62
 73
  2
  2
 55
 54
 60
 72
 72
 62
 56
 65
  1
 58
 73
  1
 58
 73
 65
 67
 68
  2
 54
 54
 62
 67
 65
 74

In [40]:
xs[0].size(0)

512

In [29]:
fit(m,md,1,opt,F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      1.977411   1.968401  



[array([1.9684])]

In [33]:
set_lrs(opt,0.001)

In [34]:
fit(m,md,1,opt,F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      1.695717   1.696457  



[array([1.69646])]

####  Now we use Pytorch to create the same RNN

In [61]:
class FirstPytorchRNN(nn.Module):
    def __init__(self,vocab_size,n_fac):
        super().__init__()
        self.e=nn.Embedding(vocab_size,n_fac)
        self.rnn=nn.RNN(n_fac,n_hidden)
        self.l_out=nn.Linear(n_hidden,vocab_size)
        
    def forward(self,*cs):
        bs=cs[0].size(0)
        h=V(torch.zeros(1,bs,n_hidden).cuda())
        inp=self.e(torch.stack(cs))
        outp,h=self.rnn(inp,h)
        return(F.log_softmax(self.l_out(outp[-1]),dim=-1))

In [62]:
m=FirstPytorchRNN(vocab_size,n_fac).cuda()
opt=optim.Adam(m.parameters(),1e-3)

In [63]:
fit(m,md,1,opt,F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      1.881922   1.85588   



[array([1.85588])]

## use GRU and LSTM from Pytorch

In [14]:
from fastai import sgdr

In [21]:
from torchtext import vocab, data
from fastai.text import *
from fastai.lm_rnn import *
from fastai.nlp import *

In [16]:
PATH='data/nietzsche/'

In [17]:
TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt  trn/  val/


In [31]:
%ls{PATH}trn

trn.txt


In [24]:
TEXT=data.Field(lower=True,tokenize=list)
bs=64;bptt=8;n_fac=42

FILES=dict(train=TRN_PATH,validation=VAL_PATH,test=VAL_PATH)

md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(942, 55, 1, 482908)

In [27]:
??repackage_var

In [26]:
??ConcatTextDataset

In [37]:
n_hidden=512

In [48]:
class MyLSTMRnn(nn.Module):
    
    def __init__(self,vocab_size,n_fac,bs,nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e=nn.Embedding(n_hidden,n_fac)
        self.rnn=nn.LSTM(n_fac,n_hidden,nl,dropout=0.5)
        self.l_out=nn.Linear(n_hidden,vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [49]:
m = MyLSTMRnn(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [50]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [51]:
fit(m, md, 2, lo.opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      1.821988   1.736199  
    1      1.713187   1.617606                              



[array([1.61761])]

In [52]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      1.529518   1.472164  
    1      1.583065   1.509109                              
    2      1.448325   1.418017                              
    3      1.612189   1.535976                              
    4      1.524319   1.469557                              
    5      1.445502   1.407873                              
    6      1.377405   1.37387                               
    7      1.603567   1.532815                              
    8      1.550545   1.497369                              
    9      1.518477   1.475922                              
    10     1.484052   1.441973                              
    11     1.439613   1.407986                              
    12     1.382698   1.377323                              
    13     1.338167   1.353011                              
    14     1.309745   1.339                                 



[array([1.339])]

In [53]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      1.294978   1.337187  
    1      1.299174   1.334979                              
    2      1.295987   1.334602                              
    3      1.294428   1.333076                              
    4      1.291541   1.330348                              
    5      1.286399   1.328944                              
    6      1.281721   1.329016                              
    7      1.287481   1.327903                              
    8      1.279981   1.325342                              
    9      1.270692   1.323214                              
    10     1.26043    1.321665                              
    11     1.257803   1.322192                              
    12     1.25446    1.320258                              
    13     1.251675   1.320098                              
    14     1.2537     1.319983                              
    15     1.256717   1.32148                       

[array([1.34666])]

In [54]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [55]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [56]:
print(get_next_n('for thos', 400))

for those thouthine's home: he his philosopsits which is, as it was only and not-cense with such, asformer effects(the german dicinatelly; not bestow:--it is has longer be handed for theworld of intellectual work in our bad. there are refording for the basis, with my solitude,--we have a discovery, apparently of the part my enammit of the people laugh our brought prevails, men--i, repulstest,[3] gay the s


### Attempt to create a dataloader from scratch

In [57]:
import spacy

In [58]:
spacy_en=spacy.load('en')

In [59]:
def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [60]:
TEXT=data.Field(sequential=True, tokenize=tokenizer, lower=True)